In [1]:
import panel as pn
import numpy as np
from pycromanager import Core
from bokeh.plotting import figure
from bokeh.events import Tap, SelectionGeometry
from bokeh.models import BoxEditTool, ColumnDataSource

pn.extension() #render panel inline

In [ ]:
core = Core()

Display and hist tab

In [ ]:
#widgets
image_button = pn.widgets.Button(name='Snap Image', button_type='primary') #panel widget
image_widget = pn.panel(figure(title='Display')) #bokeh wrapper
histo_button = pn.widgets.Button(name='Histogram', button_type='primary')
histo_widget = pn.panel(figure(title='Histogram'))

up_button    = pn.widgets.Button(name='^', button_type='primary')
left_button  = pn.widgets.Button(name='<', button_type='primary')
right_button = pn.widgets.Button(name='>', button_type='primary')
down_button  = pn.widgets.Button(name='v', button_type='primary')
coord_text   = pn.widgets.TextInput(value=str([core.get_x_position(),core.get_y_position()]))
step_size    = pn.widgets.FloatInput(name='step size', value=1., step=1e-1, start=0, end=1500)

xy_stage=core.get_xy_stage_device() #set stage to move

#MM snap image
def snap():
    core.wait_for_system()
    core.snap_image()
    tagged_image = core.get_tagged_image()
    
    return tagged_image


#on_click callbacks
def on_image_button_clicked(b):
    core.clear_roi()
    tagged_image=snap()
    
    #reshape to 3d array
    height = tagged_image.tags['Height']
    width = tagged_image.tags['Width']
    pixels = np.reshape(tagged_image.pix,
                    newshape=[height, width, tagged_image.pix.shape[0]//(height*width)])
    
    #reshape to 2d array for bokeh's image_rgba
    image = pixels.view("uint32").reshape(pixels.shape[:2])
    
    
    resize = 500/image.shape[1] #resize to have width = 500 and height scaled
    
    #create blank figure for image
    #set tooltips for displaying hover (x and y coord, image value)
    p=figure(title= 'Display',tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")],
             y_range=(image.shape[0],0),
             frame_width=500, frame_height=int(image.shape[0]*resize))
    
    p.image_rgba(image=[image],x=0, y=0, dw=image.shape[1], dh=image.shape[0])
    image_widget.object=p #update widget
                                             

def on_histo_button_clicked(b):
    tagged_image= snap()
    
    # Histogram
    hist, edges = np.histogram(tagged_image.pix, density=True, bins=64)
    y_max=max(hist) + (max(hist)/10) #give room at the top
    #set tooltips to dispaly bokeh's quad's "top" input
    p=figure(title='Histogram',tooltips=[("Value", "@top")], y_range=(0, y_max), frame_width=1280)
    
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
    fill_color="lightgrey", line_color="black")
    histo_widget.object=p #update widget
    
def on_move_up_button_clicked(b):
    core.set_relative_xy_position(xy_stage, 0,step_size.value)
    coord_text.value=str([core.get_x_position(),core.get_y_position()])
    
def on_move_left_button_clicked(b):
    core.set_relative_xy_position(xy_stage, -step_size.value,0)
    coord_text.value=str([core.get_x_position(),core.get_y_position()])

def on_move_right_button_clicked(b):
    core.set_relative_xy_position(xy_stage, step_size.value,0)
    coord_text.value=str([core.get_x_position(),core.get_y_position()])
    
def on_move_down_button_clicked(b):
    core.set_relative_xy_position(xy_stage, 0,-step_size.value)
    coord_text.value=str([core.get_x_position(),core.get_y_position()])
        

#on_click events
image_button.on_click(on_image_button_clicked)
histo_button.on_click(on_histo_button_clicked)
up_button.on_click(on_move_up_button_clicked)
left_button.on_click(on_move_left_button_clicked) 
right_button.on_click(on_move_right_button_clicked)
down_button.on_click(on_move_down_button_clicked)


#dash layout
display_app = pn.Column(
    pn.Row(
        pn.Column(
            image_button, 
            histo_button,
            step_size,
            pn.Row(up_button,align='center'),
            pn.Row(left_button, right_button, align='center'),
            pn.Row(down_button, align='center'),
            coord_text, sizing_mode='stretch_height'),
        image_widget),
    histo_widget)

ROI tab

In [ ]:
#widgets
display_button = pn.widgets.Button(name='Display', button_type='primary') #panel widget
display_widget = pn.panel(figure(name='Display')) #bokeh wrapper


roi_button   = pn.widgets.Button(name='ROI', button_type='primary')
roi_widget   = pn.panel(figure())

#MM snap image
def snap():
    core.wait_for_system()
    core.snap_image()
    tagged_image = core.get_tagged_image()
    
    return tagged_image

box_coord=[]

def box(event):
    box_coord.clear()
    box_coord.append(event.geometry)

    
#on_click callbacks
def on_display_button_clicked(b):
    core.clear_roi()
    tagged_image=snap()
    
    #reshape to 3d array
    height = tagged_image.tags['Height']
    width = tagged_image.tags['Width']
    pixels = np.reshape(tagged_image.pix,
                    newshape=[height, width, tagged_image.pix.shape[0]//(height*width)])
    
    #reshape to 2d array for bokeh's image_rgba
    image = pixels.view("uint32").reshape(pixels.shape[:2])
    
    
    resize = 500/image.shape[1] #resize to have widht = 500
    
    #create blank figure for image
    #set tooltips for displaying hover (x and y coord, image value)
    p=figure(title= 'Display',tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")],tools=["box_select"], 
             y_range= (image.shape[0],0),
             frame_width=500, frame_height=int(image.shape[0]*resize))
    
    p.image_rgba(image=[image],x=0, y=0, dw=image.shape[1], dh=image.shape[0])
    p.on_event(SelectionGeometry, box)
    display_widget.object=p #update widget
                               

def on_roi_button_clicked(b):
    if len(box_coord) != 0:
        
        x=int(box_coord[0]['x0'])
        y=int(box_coord[0]['y0'])
        dx=int(abs(x-box_coord[0]['x1']))
        dy=int(abs(y-box_coord[0]['y1']))
        
        core.clear_roi()
        core.set_roi(x,y,dx,dy)
        
        #display image
        tagged_image=snap()
    
        #reshape to 3d array
        height = tagged_image.tags['Height']
        width = tagged_image.tags['Width']
        pixels = np.reshape(tagged_image.pix,
                        newshape=[height, width, tagged_image.pix.shape[0]//(height*width)])

        #reshape to 2d array for bokeh's image_rgba
        image = pixels.view("uint32").reshape(pixels.shape[:2])
        
        #create blank figure for image
        #set tooltips for displaying hover (x and y coord, image value)
        p=figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")], frame_width=500, 
                 y_range= (image.shape[0],0),
                 frame_height=int(image.shape[0]*(500/image.shape[1])))

        p.image_rgba(image=[image],x=0, y=0, dw=image.shape[1], dh=image.shape[0])
        roi_widget.object=p #update widget
        
display_button.on_click(on_display_button_clicked)
roi_button.on_click(on_roi_button_clicked)

roi_app = pn.Row(pn.Column(display_button, display_widget), pn.Column(roi_button, roi_widget))

tabs

In [ ]:
tabs = pn.Tabs(('Display Image',display_app), ('ROI',roi_app), dynamic=True) #('Flim',flim_app),
tabs.servable() #must call servable() for it to render in server